In [1]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath('../scripts'))

# Change the current working directory to the appropriate path
os.chdir('/home/semre/telecom_user_behaviour_analysis')
# Import the required functions and modules
from src.utils import missing_values_table, connvert_bytes_to_megabytes, fix_outlier, remove_outliers
from data_loader import load_data_from_db
# Call the function to load data
query = "SELECT * FROM public.xdr_data"
df = load_data_from_db(query)

Host: localhost, Port: 5432, DB: telecom_db, User: postgres


In [2]:
# Check the columns in the DataFrame
print(df.columns)

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [3]:
missing_values_df = missing_values_table(df)
print("Missing Values in df:")
print(missing_values_df)

Your selected dataframe has 55 columns
There are 41 columns that have missing values.
Missing Values in df:
                                          mis_val  % of Total Values    Dtype
Nb of sec with 37500B < Vol UL             130254               86.8  float64
Nb of sec with 6250B < Vol UL < 37500B     111843               74.6  float64
Nb of sec with 125000B < Vol DL             97538               65.0  float64
TCP UL Retrans. Vol (Bytes)                 96649               64.4  float64
Nb of sec with 31250B < Vol DL < 125000B    93586               62.4  float64
Nb of sec with 1250B < Vol UL < 6250B       92894               61.9  float64
Nb of sec with 6250B < Vol DL < 31250B      88317               58.9  float64
TCP DL Retrans. Vol (Bytes)                 88146               58.8  float64
HTTP UL (Bytes)                             81810               54.5  float64
HTTP DL (Bytes)                             81474               54.3  float64
Avg RTT DL (ms)                   

In [7]:
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans

# Define the engagement and experience columns
engagement_columns = [
    'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)', 'Gaming UL (Bytes)'
]

experience_columns = [
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
    'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)'
]

# Step 1: Cluster data using KMeans for engagement and experience scores
kmeans_engagement = KMeans(n_clusters=2, random_state=42)
kmeans_experience = KMeans(n_clusters=2, random_state=42)

# Fit KMeans and assign clusters for engagement and experience
df['engagement_cluster'] = kmeans_engagement.fit_predict(df[engagement_columns])
df['experience_cluster'] = kmeans_experience.fit_predict(df[experience_columns])

# Get the centroids for least engaged and worst experience clusters
least_engaged_centroid = kmeans_engagement.cluster_centers_[0]  # Update index for least engaged cluster
worst_experience_centroid = kmeans_experience.cluster_centers_[0]  # Update index for worst experience cluster

# Calculate Euclidean distances for engagement and experience scores
df['engagement_score'] = pairwise_distances(df[engagement_columns], [least_engaged_centroid]).flatten()
df['experience_score'] = pairwise_distances(df[experience_columns], [worst_experience_centroid]).flatten()

# Step 2: Calculate satisfaction score as the average of engagement and experience scores
df['satisfaction_score'] = df[['engagement_score', 'experience_score']].mean(axis=1)

# Step 3: Report top 10 satisfied customers
top_10_satisfied_customers = df[['MSISDN/Number', 'satisfaction_score']].nlargest(10, 'satisfaction_score')
print(top_10_satisfied_customers)

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values